In [ ]:
# import bw2io as bi

In [ ]:
# bi.restore_project_directory('/Users/timodiepers/Documents/Coding/brightway2-project-bw25_premise_background_v2-backup.26-March-2024-01-40PM.tar.gz', overwrite_existing=True, project_name='bw25_premise')

In [ ]:
import bw2data as bd

In [ ]:
bd.projects.set_current('bw25_premise')

In [ ]:
db_2020 = bd.Database("db_2020")
db_2030 = bd.Database("db_2030")
db_2040 = bd.Database("db_2040")

In [ ]:
del bd.databases['foreground']

In [ ]:
ev_background = [x for x in db_2020 if (x['name'] == 'transport, passenger car, electric')][0]
ev_background

In [ ]:
[e for e in ev_background.exchanges()]

In [ ]:
foreground = bd.Database("foreground")
foreground.write({})

In [ ]:
ev = ev_background.copy(name="copy of transport, passenger car, electric")

In [ ]:
ev['database'] = 'foreground'

In [ ]:
ev.save()

In [ ]:
[e for e in ev.exchanges()]

In [ ]:
from bw_temporalis import TemporalDistribution
import numpy as np

# defining Temporal Distributions of system
td_production = TemporalDistribution(
            date=np.array([-1], dtype='timedelta64[Y]'), #2023
            amount=np.array([1]))

td_use_phase = TemporalDistribution( #to be further refined based on use pattern
                    date=np.array([1, 6, 11, 16], dtype='timedelta64[Y]'), #2025, 2030, 2035, 2040
                    amount=np.array([0.25, 0.25, 0.25, 0.25]))

td_eol = TemporalDistribution(
            date=np.array([21], dtype='timedelta64[Y]'), #2045
            amount=np.array([1]))

td_eol_from_production_time = TemporalDistribution(
            date=np.array([22], dtype='timedelta64[Y]'), #2045, counting from 2023
            amount=np.array([1]))

In [ ]:
#add TD to electricity and car production
for exc in ev.exchanges():
    if "market for battery, Li-ion"in exc.input["name"]: #battery production
        exc["temporal_distribution"]= td_production
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "market for passenger car, electric, without battery"in exc.input["name"]: # foreground car production
        exc["temporal_distribution"]= td_production
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "market group for electricity, low voltage" in exc.input["name"]: #electricity while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "market for used Li-ion battery" in exc.input["name"]:   #battery recycling
        exc["temporal_distribution"]= td_eol
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "wear emissions, passenger car" in exc.input["name"]: #brake, road and tyre wear while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
    
    elif "market for maintenance, passenger car" in exc.input["name"]: #car maintenance while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
            
    elif exc.input["name"] == "market for road": # road usage while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
    
    elif exc.input["name"] == exc.output["name"]: #skip production exchange
        continue

    else:
        print("no TD added to ", {exc.input["name"]})

    # car recycling is modelled at the level of car production, and TD is added there

In [ ]:
method = ('IPCC 2021 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

In [ ]:
from datetime import datetime

database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    # "cutoff39": datetime.strptime("2020", "%Y"), # all databases need to have a corresponding time
    "foreground": "dynamic",  # flag databases that should be temporally distributed with "dynamic"
}

In [ ]:
from timex_lca import MedusaLCA
mlca = MedusaLCA({ev.key: 1}, method, None, database_date_dict) #aborted after 100 min for max_calc = 5000


In [ ]:
mlca.build_timeline()

In [ ]:
mlca.build_datapackage()
mlca.lci()
mlca.lcia()

In [ ]:
mlca.create_labelled_technosphere_dataframe()

In [ ]:

mlca.create_labelled_dynamic_biosphere_dataframe()

In [ ]:
co2_flows = [flow for flow in bd.Database('biosphere3') if 'Carbon dioxide' in flow['name'] and "air" in flow['categories']]
co2_flows

In [ ]:
co2_flows_codes = [flow.key[1] for flow in co2_flows]

In [ ]:
co2_flows_codes

In [ ]:
mlca.dynamic_inventory["d6235194-e4e6-4548-bfa3-ac095131aef4"]

In [ ]:
mlca.plot_dynamic_inventory(bio_flows=co2_flows_codes)